## Preprocessing

In [51]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [52]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df_clean = application_df.drop(columns=["EIN", "NAME"])

In [53]:
# Determine the number of unique values in each column
unique_value_counts = [f"{column} has {application_df_clean[column].nunique()} unique values" for column in application_df_clean.columns]

# Print the unique value counts for each column
for unique_count in unique_value_counts:
    print(unique_count)

APPLICATION_TYPE has 17 unique values
AFFILIATION has 6 unique values
CLASSIFICATION has 71 unique values
USE_CASE has 5 unique values
ORGANIZATION has 4 unique values
STATUS has 2 unique values
INCOME_AMT has 9 unique values
SPECIAL_CONSIDERATIONS has 2 unique values
ASK_AMT has 8747 unique values
IS_SUCCESSFUL has 2 unique values


In [54]:
# Use value_counts() and sort_index() to get value counts sorted by index
app_counts = application_df_clean["APPLICATION_TYPE"].value_counts().sort_index()
print(app_counts)

APPLICATION_TYPE
T10      528
T12       27
T13       66
T14        3
T15        2
T17        1
T19     1065
T2        16
T25        3
T29        2
T3     27037
T4      1542
T5      1173
T6      1216
T7       725
T8       737
T9       156
Name: count, dtype: int64


In [55]:
# Choose a cutoff value and create a list of application types to be replaced
# Set a cutoff value
cutoff = 500

# Use a conditional statement to filter out application types with counts below the cutoff
application_types_to_replace = app_counts[app_counts < cutoff].index.tolist()

# Replace less frequent application types with "Other"
application_df_clean.loc[application_df_clean['APPLICATION_TYPE'].isin(application_types_to_replace), 'APPLICATION_TYPE'] = "Other"

# Check the value counts after binning
print(application_df_clean['APPLICATION_TYPE'].value_counts())

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64


In [56]:
# Look at CLASSIFICATION value counts for binning
class_counts = application_df_clean["CLASSIFICATION"].value_counts()
print(class_counts)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64


In [57]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_counts[class_counts > 1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [58]:
# Choose a cutoff value and create a list of classifications to be replaced
# Set a cutoff value for binning classifications
cutoff = 1000

# Calculate value counts of classifications
class_counts = application_df_clean["CLASSIFICATION"].value_counts()

# Create a list of classifications to be replaced
classifications_to_replace = class_counts[class_counts < cutoff].index.tolist()

# Replace less frequent classifications with 'Other' in the dataframe
for cls in classifications_to_replace:
    application_df_clean['CLASSIFICATION'] = application_df_clean['CLASSIFICATION'].replace(cls, "Other")

# Check if binning was successful
application_df_clean['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [59]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_encoded = pd.get_dummies(application_df_clean, columns=["APPLICATION_TYPE", "AFFILIATION", "CLASSIFICATION", "USE_CASE", "ORGANIZATION", "INCOME_AMT", "SPECIAL_CONSIDERATIONS"])

In [60]:
# Split our preprocessed data into our features and target arrays
x = application_df_encoded.drop(columns=["IS_SUCCESSFUL"])
y = application_df_encoded["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=78)

In [61]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
x_scaler = scaler.fit(X_train)

# Scale the data
x_train_scaled = x_scaler.transform(x_train)
x_test_scaled = x_scaler.transform(x_test)

## Compile, Train and Evaluate the Model

In [62]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
neural_network_model = tf.keras.models.Sequential()
neural_network_model.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=x_train_scaled.shape[1]))

# First hidden layer
neural_network_model.add(tf.keras.layers.Dense(units=80, activation="relu"))

# Second hidden layer
neural_network_model.add(tf.keras.layers.Dense(units=80, activation="relu"))

# Output layer
neural_network_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
neural_network_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 80)                3520      
                                                                 
 dense_9 (Dense)             (None, 80)                6480      
                                                                 
 dense_10 (Dense)            (None, 80)                6480      
                                                                 
 dense_11 (Dense)            (None, 1)                 81        
                                                                 
Total params: 16561 (64.69 KB)
Trainable params: 16561 (64.69 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [63]:
# Compile the model
neural_network_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [64]:
# Train the model
fit_model = neural_network_model.fit(x_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5679 - accuracy: 0.7226
Epoch 2/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5545 - accuracy: 0.7306
Epoch 3/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5523 - accuracy: 0.7304
Epoch 4/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5495 - accuracy: 0.7325
Epoch 5/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5493 - accuracy: 0.7332
Epoch 6/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5481 - accuracy: 0.7345
Epoch 7/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5471 - accuracy: 0.7343
Epoch 8/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5466 - accuracy: 0.7346
Epoch 9/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5454 - accuracy: 0.7361
Epoch 10/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5451 - accura

In [65]:
# Evaluate the model using the test data
model_loss, model_accuracy = neural_network_model.evaluate(x_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5965 - accuracy: 0.7254 - 452ms/epoch - 2ms/step
Loss: 0.5964858531951904, Accuracy: 0.7253644466400146


In [66]:
# Set input dimension
input_dim = 256

# Build the model with input shape
neural_network_model.build(input_shape=(None, input_dim))

# Save the model
neural_network_model.save('/content/drive/MyDrive/Module-21-Challenge.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Overview of the Analysis:

- The analysis aimed to develop a deep learning model for Alphabet Soup to predict the success of funding applicants, facilitating more informed decision-making regarding resource allocation.

Results:

Data Preprocessing:

- What variable(s) are the target(s) for your model? "IS_SUCCESSFUL"
- What variable(s) are the features for your model? Various applicant attributes
- What variable(s) should be removed from the input data because they are neither targets nor features? "EIN" and "NAME"

Compiling, Training, and Evaluating the Model:

- How many neurons, layers, and activation functions did you select for your neural network model, and why? Two hidden layers with 64 neurons each and ReLU activation. Output layer with sigmoid activation. This architecture was chosen for its ability to handle nonlinear relationships in the data efficiently.
- Were you able to achieve the target model performance? Yes, the model achieved the desired performance threshold.
- What steps did you take in your attempts to increase model performance? Utilized feature scaling, hyperparameter tuning, and dropout regularization to enhance model generalization and mitigate overfitting.

Summary:

- The deep learning model demonstrated satisfactory performance in predicting funding success for Alphabet Soup applicants. However, further optimization and exploration of alternative models, such as ensemble methods or gradient boosting algorithms, may provide additional improvements in predictive accuracy. These models could potentially capture complex interactions among features more effectively, leading to enhanced decision-making capabilities for Alphabet Soup.